
# TOV + Frame Dragging (Slow Rotation, Demo)

This notebook integrates the TOV equations for a toy gamma-law EOS, then solves the first-order frame-dragging equation for \(\bar{\omega}(r)=\Omega-\omega(r)\). We set \(\Omega=1\) (scaling choice), so the **moment of inertia** is simply \(I=J\).

**Equations:**

```math
\frac{dM}{dr} = 4\pi r^2 E,\qquad
\frac{dP}{dr} = -\frac{(E+P)(M+4\pi r^3 P)}{r(r-2M)},\qquad
\frac{d\Phi}{dr} = -\frac{1}{E+P}\frac{dP}{dr}.
```

Gamma-law EOS (demo):

```math
P = K\,\rho^{\gamma},\qquad
E = \rho + \frac{P}{\gamma-1}.
```

Frame-dragging (regular at center):

```math
\frac{d}{dr}\Bigl(r^4 j(r) \frac{d\bar{\omega}}{dr}\Bigr) + 4 r^3 \frac{dj}{dr} \, \bar{\omega} = 0,
\quad
j(r) = e^{-\Phi(r)}\sqrt{1-\frac{2M(r)}{r}}.
```

Exterior: \(\bar{\omega}(r)=2J/r^3\). With \(\Omega=1\), we identify \(I=J\).


In [ ]:

import numpy as np
import matplotlib.pyplot as plt


In [ ]:

def eos_gamma_law_from_P(P, K, gamma):
    """Given pressure P, return (rho, E) for a gamma-law EOS:
    P = K rho^gamma,  E = rho + P/(gamma-1)."""
    P = max(P, 1e-60)
    rho = (P / K)**(1.0/gamma)
    E = rho + P/(gamma - 1.0)
    return rho, E


In [ ]:

def tov_rhs(r, y, K, gamma):
    """RHS for TOV+Phi: y = [M, P, Phi]."""
    M, P, Phi = y
    if P <= 0.0:
        return np.array([0.0, 0.0, 0.0])
    rho, E = eos_gamma_law_from_P(P, K, gamma)
    dMdr = 4.0*np.pi * r**2 * E
    denom = r*(r - 2.0*M) if r > 0 else np.inf
    dPdr = -(E + P) * (M + 4.0*np.pi*r**3 * P) / denom
    dPhidr = -(1.0/(E + P)) * dPdr if (E+P)>0 else 0.0
    return np.array([dMdr, dPdr, dPhidr])


In [ ]:

def rk4_step(fun, r, y, h, *args):
    k1 = fun(r, y, *args)
    k2 = fun(r + 0.5*h, y + 0.5*h*k1, *args)
    k3 = fun(r + 0.5*h, y + 0.5*h*k2, *args)
    k4 = fun(r + h, y + h*k3, *args)
    return y + (h/6.0)*(k1 + 2*k2 + 2*k3 + k4)

def integrate_tov(Pc, K, gamma, r_max=50.0, h=1e-3):
    """Integrate TOV outward from small r to P=0 (surface)."""
    r = h
    rho_c, E_c = eos_gamma_law_from_P(Pc, K, gamma)
    M = (4.0/3.0)*np.pi * E_c * r**3
    P = Pc
    Phi = 0.0
    rs, Ms, Ps, Phis = [0.0, r], [0.0, M], [Pc, P], [Phi, Phi]
    while r < r_max and P > 0.0:
        y = np.array([M, P, Phi])
        y_next = rk4_step(tov_rhs, r, y, h, K, gamma)
        r_next = r + h
        M, P, Phi = y_next
        r = r_next
        rs.append(r); Ms.append(M); Ps.append(P); Phis.append(Phi)
    rs = np.array(rs); Ms = np.array(Ms); Ps = np.array(Ps); Phis = np.array(Phis)
    idx_surf = np.where(Ps<=0.0)[0]
    if len(idx_surf)==0:
        idx_surf = len(Ps)-1
    else:
        idx_surf = idx_surf[0]
    if idx_surf>0:
        r1, r2 = rs[idx_surf-1], rs[idx_surf]
        p1, p2 = Ps[idx_surf-1], Ps[idx_surf]
        if p2 != p1:
            r_surf = r1 + (0 - p1)*(r2 - r1)/(p2 - p1)
        else:
            r_surf = rs[idx_surf]
        M_surf = np.interp(r_surf, rs, Ms)
        Phi_surf = np.interp(r_surf, rs, Phis)
        mask = rs <= r_surf
        rs, Ms, Ps, Phis = rs[mask], Ms[mask], Ps[mask], Phis[mask]
        rs[-1] = r_surf; Ms[-1] = M_surf; Ps[-1] = 0.0; Phis[-1] = Phi_surf
    return dict(r=rs, M=Ms, P=Ps, Phi=Phis)


In [ ]:

def fd_rhs(r, y, r_arr, M_arr, Phi_arr):
    """First-order system for bar{omega}:
    y = [barw, z], z = d(barw)/dr.
    d/dr (r^4 j z) + 4 r^3 (dj/dr) barw = 0.
    => d(barw)/dr = z
       d z /dr = [ -4 r^3 (dj/dr) barw - d(r^4 j)/dr * z ] / (r^4 j)
    """
    barw, z = y
    # Interpolate background:
    M = np.interp(r, r_arr, M_arr)
    Phi = np.interp(r, r_arr, Phi_arr)
    j = np.exp(-Phi) * np.sqrt(max(1e-16, 1.0 - 2.0*M/max(r,1e-12)))
    # dj/dr via small centered difference:
    dr = 1e-5*max(1.0, r)
    r_m = max(1e-8, r - dr)
    r_p = r + dr
    def j_of(rr):
        Mm = np.interp(rr, r_arr, M_arr)
        Pm = np.interp(rr, r_arr, Phi_arr)
        return np.exp(-Pm) * np.sqrt(max(1e-16, 1.0 - 2.0*Mm/max(rr,1e-12)))
    jp = j_of(r_p); jm = j_of(r_m)
    djdr = (jp - jm)/(r_p - r_m)
    d_r4j_dr = 4.0*r**3 * j + r**4 * djdr
    denom = max(1e-30, r**4 * j)
    dbarw_dr = z
    dz_dr = ( -4.0*r**3 * djdr * barw - d_r4j_dr * z ) / denom
    return np.array([dbarw_dr, dz_dr])

def integrate_frame_dragging(bg, barw_center=1.0, r_eps=1e-4, h=1e-3):
    r_arr, M_arr, Phi_arr = bg['r'], bg['M'], bg['Phi']
    R = r_arr[-1]
    r = max(r_eps, r_arr[1])
    barw = barw_center
    z = 0.0
    rs = [r]; barws = [barw]; zs = [z]
    def rk4_step_fd(r, y, h):
        k1 = fd_rhs(r, y, r_arr, M_arr, Phi_arr)
        k2 = fd_rhs(r + 0.5*h, y + 0.5*h*k1, r_arr, M_arr, Phi_arr)
        k3 = fd_rhs(r + 0.5*h, y + 0.5*h*k2, r_arr, M_arr, Phi_arr)
        k4 = fd_rhs(r + h, y + h*k3, r_arr, M_arr, Phi_arr)
        return y + (h/6.0)*(k1 + 2*k2 + 2*k3 + k4)
    while r < R:
        y = np.array([barw, z])
        y_next = rk4_step_fd(r, y, h)
        r_next = r + h
        r, barw, z = r_next, y_next[0], y_next[1]
        rs.append(r); barws.append(barw); zs.append(z)
    rs = np.array(rs); barws = np.array(barws); zs = np.array(zs)
    R = rs[-1]
    barw_R = barws[-1]
    J = 0.5 * barw_R * R**3  # from exterior barw = 2J/r^3, with Omega=1
    I = J
    return dict(r=rs, barw=barws, z=zs, J=J, I=I)


In [ ]:

# Demo parameters (geometric units)
K = 100.0     # toy polytropic constant
gamma = 2.0   # Gamma-law index
Pc = 1e-3     # central pressure

# Integrate TOV
bg = integrate_tov(Pc, K, gamma, r_max=100.0, h=1e-3)
r, M, P, Phi = bg['r'], bg['M'], bg['P'], bg['Phi']
R, Mtot = r[-1], M[-1]

# Integrate frame dragging (bar{omega})
fd = integrate_frame_dragging(bg, barw_center=1.0, r_eps=1e-4, h=1e-3)
r_fd, barw = fd['r'], fd['barw']
J, I = fd['J'], fd['I']

print(f"Surface R ~ {R:.6g}, Mass M ~ {Mtot:.6g}")
print(f"Moment of inertia I ~ {I:.6g} (with Omega=1), J ~ {J:.6g}")

# Plots
plt.figure()
plt.plot(r, M)
plt.xlabel('r'); plt.ylabel('M(r)'); plt.title('Enclosed Mass'); plt.show()

plt.figure()
plt.plot(r, P)
plt.xlabel('r'); plt.ylabel('P(r)'); plt.title('Pressure'); plt.yscale('log'); plt.show()

plt.figure()
plt.plot(r_fd, barw)
plt.xlabel('r'); plt.ylabel('bar{omega}(r)'); plt.title('Frame Dragging: bar{omega}(r) (Omega=1)'); plt.show()
